In [ ]:
from coincidence_matching import match_parts, get_export_id_types, match_parts_dict
from zipfile import ZipFile
import json
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm
from automate import Part
import numpy as np
import meshplot as mp
from collections import Counter
import torch

from brepmatching.data import BRepMatchingDataset

In [ ]:
import os
from pathlib import Path
str(Path.home().joinpath('.config','onshapecreds.json'))

In [ ]:
gdrive_f1 = "https://drive.google.com/file/d/1O7FajIOF5YT_1zW337pm6XE_B2gEhUUc"
gdrive_f2 = "https://drive.google.com/file/d/1OK4qyuUfSXqA-wd64p_f2pG0rxeeqXGM"
gdrive_f3 = "https://drive.google.com/file/d/1NnUBCkIUrYBTXvEC8IXGE4V2UNDTG6T6"

In [ ]:
def gdrive_wget(FILEID, FILENAME):
    return f"wget --load-cookies /tmp/cookies.txt \"https://docs.google.com/uc?export=download&confirm=" + \
        "$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate" + \
        " 'https://docs.google.com/uc?export=download&id={FILEID}' -O- | sed -rn 's/.*confirm=([0-9A-" + \
        "Za-z_]+).*/\1\n/p')&id={FILEID}\" -O {FILENAME} && rm -rf /tmp/cookies.txt"
def url_to_wget(url, dest):
    url = url.split('/d/')[-1]
    return gdrive_wget(url, dest)

In [ ]:
from zipfile import ZipFile
import pandas as pd
from tqdm import tqdm
from apikey.onshape import Onshape
from pathlib import Path

api = Onshape(stack='https://cad.onshape.com', creds=str(Path.home().joinpath('.config','onshapecreds.json')), logging=False)

with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    variations[:100]
    variations = variations[variations.fail == 0]

    failures = []
    written = set()
    with ZipFile('../data/redownload.zip','w') as wzf:
        for i in tqdm(range(len(variations))):
            d = variations.iloc[i]

            response_orig = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_orig}/e/{d.eid}/parasolid', query={'includeExportIds':True})
            response_var = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})

            if response_orig.status_code != 200 or response_var.status_code != 200:
                failures.append(d)
                continue
            
            orig_path = 'data/BrepsWithReference/' + d.ps_orig
            var_path = 'data/BrepsWithReference/' + d.ps_var

            if orig_path not in written:
                with wzf.open(orig_path, 'w') as f:
                    f.write(response_orig.text.encode('utf-8'))
                written.add(orig_path)
            if var_path not in written:
                with wzf.open(var_path, 'w') as f:
                    f.write(response_var.text.encode('utf-8'))
                written.add(var_path)

In [ ]:
os.listdir('../../../brepmatching/Geo.zip')

In [ ]:
import pickle
with open('../../../brepmatching/GeoRedownload.zip-failures.pickle','rb') as f:
    failures = pickle.load(f)
    

In [ ]:
failures[0].link_var

In [ ]:
zf =  ZipFile('../../../brepmatching/Geo.zip','r') 

In [ ]:
with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
    variations = pd.read_csv(f)

In [ ]:
allnames = set(zf.namelist())

In [ ]:
zf.close()

In [ ]:
var_exists = []
missing = []
for p in 'data/baseline/' + variations[variations.fail == 0].baselineNew:
    if not p in allnames:
        missing.append(p)
len(missing)

In [ ]:
missing

In [ ]:
len(variations[variations.fail == 0])

In [ ]:
variations.baselineOrig.iloc[1]

In [ ]:
'data/baseline/baselineOrig_d65b13c6430b30819cb6b169_ab94f8bbfa837d13accf82d0_5dcc8a4afa039812c247cda8_default_jjeeiV2.x_t' in allnames

In [ ]:
not_failed = variations[variations.fail == 0]

In [ ]:
not_failed[not_failed.baselineNew == ' ']

In [ ]:
missing_data = []
has_missing_export_id = []
missing_match_count = []
has_missing_matches = []

#data_path = '../../../brepmatching/Geo.zip'
data_path = '../data/GeoV2FullRunBaseline.zip'

with ZipFile(data_path,'r') as zf:
    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)
    
    data = variations
    data = data[(data.fail == 0) & (data.baselineNew != ' ') ]
    
    all_names = set(zf.namelist())
    
    for i in tqdm(range(len(data))):
        d = data.iloc[i]
        
        original_path = 'data/BrepsWithReference/' + d.ps_orig
        variation_path = 'data/BrepsWithReference/' + d.ps_var
        
        baseline_original_path = 'data/baseline/' + d.baselineOrig
        baseline_var_path = 'data/baseline/' + d.baselineNew
        
        path_gt_match = 'data/Matches/' + d.matchFile
        path_bl_match = 'data/baseline/' + d.baselineMatch
        
        data_exists = (original_path in all_names) and \
            (variation_path in all_names) and \
            (baseline_original_path in all_names) and \
            (baseline_var_path in all_names) and \
            (path_gt_match in all_names) and \
            (path_bl_match in all_names)
        
        # Verify that all data files are in the archive
        if not data_exists:
            missing_data.append(i)
            continue
        
        # Load parts and match files
        with zf.open(original_path,'r') as f:
            original_part_data = f.read().decode('utf-8')
        with zf.open(variation_path,'r') as f:
            variation_part_data = f.read().decode('utf-8')
        with zf.open(baseline_original_path, 'r') as f:
            baseline_original_data = f.read().decode('utf-8')
        with zf.open(baseline_var_path, 'r') as f:
            baseline_var_data = f.read().decode('utf-8')
        with zf.open(path_gt_match, 'r') as f:
            gt_match = json.load(f)
        with zf.open(path_bl_match, 'r') as f:
            baseline_match = json.load(f)
            
        
        original_ids = get_export_id_types(original_part_data)
        variation_ids = get_export_id_types(variation_part_data)
        
        bl_original_ids = get_export_id_types(baseline_original_data)
        bl_variation_ids = get_export_id_types(baseline_var_data)
        
        # Verify that all referenced export_ids exist in the corresponding files
        export_id_missing = False
        for v in gt_match.values():
            orig_export_id = v['val1']
            var_export_id = v['val2']
            
            if orig_export_id not in original_ids or var_export_id not in variation_ids:
                export_id_missing = True
        
        for v in baseline_match.values():
            orig_export_id = v['val1']
            var_export_id = v['val2']
            
            if orig_export_id not in bl_original_ids or var_export_id not in bl_variation_ids:
                export_id_missing = True
        
        if export_id_missing:
            has_missing_export_id.append(i)
            continue
        
        # Check that exact matching the baseline variation to the primary variation always works
        
        var_matching = match_parts(baseline_var_data, variation_part_data, True)
        var_matching_dict = match_parts_dict(baseline_var_data, variation_part_data, True)
        
        n_faces_variation = len([k for k,v in variation_ids.items() if v == 'PK_CLASS_face'])
        n_faces_baseline_variation = len([k for k,v in bl_variation_ids.items() if v == 'PK_CLASS_face'])
        
        n_edges_variation = len([k for k,v in variation_ids.items() if v == 'PK_CLASS_edge'])
        n_edges_baseline_variation = len([k for k,v in bl_variation_ids.items() if v == 'PK_CLASS_edge'])
        
        n_vertices_variation = len([k for k,v in variation_ids.items() if v == 'PK_CLASS_vertex'])
        n_vertices_baseline_variation = len([k for k,v in bl_variation_ids.items() if v == 'PK_CLASS_vertex'])
        
        #assert(len(var_matching.face_matches) == n_faces_variation)
        #assert(len(var_matching.face_matches) == n_faces_baseline_variation)
        #assert(len(var_matching.edge_matches) == n_edges_variation)
        #assert(len(var_matching.edge_matches) == n_edges_baseline_variation)
        #assert(len(var_matching.vertex_matches) == n_vertices_variation)
        #assert(len(var_matching.vertex_matches) == n_vertices_baseline_variation)
        
        good_topos = ['PK_CLASS_face', 'PK_CLASS_edge', 'PK_CLASS_vertex']
        
        n_matches_missing = 0
        for v in baseline_match.values():
            var_export_id = v['val2']
            if bl_variation_ids[var_export_id] in good_topos:
                if var_export_id not in var_matching_dict:
                    n_matches_missing += 1
        
        missing_match_count.append(n_matches_missing)
        
        if n_matches_missing > 0:
            has_missing_matches.append(i)
            

In [ ]:
mmc = np.array(missing_match_count)

In [ ]:
len(mmc)

In [ ]:
(mmc > 0).sum()

In [ ]:
mmc.sum()

In [ ]:
sum(has_missing_matches)

In [ ]:
print(len(var_matching.edge_matches) == n_edges_variation)
print(len(var_matching.edge_matches) == n_edges_baseline_variation)


In [ ]:
variation_part = Part(variation_part_data)
baseline_variation_part = Part(baseline_var_data)

In [ ]:
bl_missing_edge_indices = [i for i,e in enumerate(baseline_variation_part.brep.nodes.edges) if e.export_id not in var_matching_dict]

In [ ]:
missing_edge_indices = [i for i,e in enumerate(variation_part.brep.nodes.edges) if e.export_id not in var_matching_dict.values()]

In [ ]:
missing_edge_indices

In [ ]:
baseline_variation_part.brep.nodes.edges[8].function

In [ ]:
bl_missing_edge_indices

In [ ]:
e2t = variation_part.mesh_topology.edge_to_topology
V = variation_part.mesh.V
F = variation_part.mesh.F
E = []
for i in range(e2t.shape[0]):
    for j in range(e2t.shape[1]):
        if e2t[i,j] in missing_edge_indices:
            E.append((F[i,(j-1)%3], F[i,j]))
E = np.array(E)

In [ ]:
e2t = baseline_variation_part.mesh_topology.edge_to_topology
V_b = baseline_variation_part.mesh.V
F_b = baseline_variation_part.mesh.F
E_b = []
for i in range(e2t.shape[0]):
    for j in range(e2t.shape[1]):
        if e2t[i,j] in bl_missing_edge_indices:
            E_b.append((F[i,(j-1)%3], F[i,j]))
E_b = np.array(E_b)

In [ ]:
np.savez('meshes.nzp',V=V,F=F,E=E,V_b=V_b,F_b=F_b,E_b=E_b)

In [ ]:
np.save('E.npy',E)

In [ ]:
mp.plot(variation_part.mesh.V, variation_part.mesh.F)

In [ ]:
mp.plot(baseline_variation_part.mesh.V, baseline_variation_part.mesh.F)

In [ ]:
variation_part.mesh.V.shape

In [ ]:
import igl

In [ ]:
igl.write_obj('baseline_var.obj', baseline_variation_part.mesh.V, baseline_variation_part.mesh.F)
igl.write_obj('var.obj', variation_part.mesh.V, variation_part.mesh.F)

In [ ]:
n_edges_baseline_variation

In [ ]:
len(var_matching.edge_matches)

In [ ]:
original_export_types = get_export_id_types(original_part_data)

In [ ]:
match_ids_original = [v['val1'] for v in gt_match.values()]
match_ids_variation = [v['val2'] for v in gt_match.values()]
bl_match_ids_original = [v['val1'] for v in baseline_match.values()]
bl_match_ids_variation = [v['val2'] for v in baseline_match.values()]



In [ ]:
all([x in original_export_types for x in match_ids_original])

In [ ]:
for v in gt_match.values():
    orig_export_id = v['val1']
    var_export_id = v['val2']

    if orig_export_id not in original_part_data:
        print(f'missing orig: {orig_export_id}')
    
    if var_export_id not in variation_part_data:
        print(f'missing var: {var_export_id}')

for v in baseline_match.values():
    bl_orig_export_id = v['val1']
    bl_var_export_id = v['val2']

    if bl_orig_export_id not in baseline_original_data:
        print(f'missing baseline orig: {bl_orig_export_id}')
    
    if bl_var_export_id not in baseline_var_data:
        print(f'missing baseline var {bl_var_export_id}')

In [ ]:
#with ZipFile('../data/TopoandGeoV2FullRunWith100SamplesBaseline.zip','r') as zf:

#with ZipFile('../data/GeoV2FullRunBaseline.zip','r') as zf:

with ZipFile('../../../brepmatching/Geo.zip','r') as zf:


    with zf.open('data/baseline/allVariationsWithBaseline.csv','r') as f:
        variations = pd.read_csv(f)

    data = variations[:100]
    data = data[(data.fail == 0) & (data.baselineNew != ' ') ]


#with ZipFile('../../../brepmatching/GeoRedownload.zip','r') as zf:
    
    incomplete = []
    incompatible = []
    diff_geo = []

    m_f = []
    m_e = []
    m_v = []

    A = []
    B = []
    M = []

    for i in tqdm(range(len(data))):
        d = data.iloc[i]
        path_a = 'data/BrepsWithReference/' + d.ps_orig
        path_b = 'data/baseline/' + d.baselineOrig

        path_c = 'data/BrepsWithReference/' + d.ps_var
        path_d = 'data/baseline/' + d.baselineNew
        
        path_gt_match = 'data/Matches/' + d.matchFile
        path_bl_match = 'data/baseline/' + d.baselineMatch

        with zf.open(path_a,'r') as f:
            part_data_a = f.read().decode('utf-8')
        with zf.open(path_b,'r') as f:
            part_data_b = f.read().decode('utf-8')
        with zf.open(path_c,'r') as f:
            part_data_c = f.read().decode('utf-8')
        with zf.open(path_d,'r') as f:
            part_data_d = f.read().decode('utf-8')
        with zf.open(path_gt_match,'r') as f:
            gt_match = json.load(f)
        with zf.open(path_bl_match, 'r') as f:
            bl_match = json.load(f)

        p_a = Part(part_data_a)
        p_b = Part(part_data_b)
        p_c = Part(part_data_c)
        p_d = Part(part_data_d)

        parts_have_same_num = len(p_a.brep.nodes.faces) == len(p_b.brep.nodes.faces) and len(p_a.brep.nodes.edges) == len(p_b.brep.nodes.edges) and len(p_a.brep.nodes.vertices) == len(p_b.brep.nodes.vertices)

        matching = match_parts(part_data_a, part_data_b, True)
        complete_matching = len(matching.face_matches) == len(p_a.brep.nodes.faces) and len(matching.edge_matches) == len(p_a.brep.nodes.edges) and len(matching.vertex_matches) == len(p_a.brep.nodes.vertices)

        n_face_matches_missing = -len(matching.face_matches) + len(p_a.brep.nodes.faces)
        n_edge_matches_missing = -len(matching.edge_matches) + len(p_a.brep.nodes.edges)
        n_vertex_matches_missing =-len(matching.vertex_matches) + len(p_a.brep.nodes.vertices)

        m_f.append(n_face_matches_missing)
        m_e.append(n_edge_matches_missing)
        m_v.append(n_vertex_matches_missing)

        exact_geo = (p_a.mesh.V == p_b.mesh.V).all()

        if not parts_have_same_num:
            incompatible.append(i)
        if not complete_matching:
            incomplete.append(i)

        if not exact_geo:
            diff_geo.append(i)

        A.append(p_a)
        B.append(p_b)
        M.append(matching)


        # Exact matching of originals seems to work (and the tesselations are mostly identical!)
        # Let's see how well we match the versions with variations:

        var_matching = match_parts(part_data_c, part_data_d, True)

        id_classes_a = get_export_id_types(part_data_a)
        id_classes_b = get_export_id_types(part_data_b)
        id_classes_c = get_export_id_types(part_data_c)
        id_classes_d = get_export_id_types(part_data_d)

        good_classes = ['PK_CLASS_face', 'PK_CLASS_edge', 'PK_CLASS_vertex']

        #gt_topo_matches = [(v['val1'],v['val1']) for k,v in gt_match.items() if id_classes_a[v['val1']] in good_classes and id_classes_c[v['val2']] in good_classes]
        bl_topo_matches = [(v['val1'],v['val1']) for k,v in bl_match.items() if id_classes_b[v['val1']] in good_classes and id_classes_d[v['val2']] in good_classes]

        


In [ ]:
'JF4' in part_data_d

In [ ]:
data

In [ ]:
d.link_var

In [ ]:
part_b = Part(part_data_b)

In [ ]:
len([e.export_id for e in part_b.brep.nodes.edges])

In [ ]:
len([k for k,v in id_classes_b.items() if v == 'PK_CLASS_edge'])

In [ ]:
id_classes_b

In [ ]:
d.link_var

In [ ]:
data.baselineMatch

In [ ]:
from apikey.onshape import Onshape

In [ ]:
response = api.request(method='get', path=f'/api/partstudios/d/{d.did}/m/{d.mv_var}/e/{d.eid}/parasolid', query={'includeExportIds':True})#, query={'partIds':['JFD'],'includeExportIds':True})

In [ ]:
response.text == part_data_b

In [ ]:
d.mv_var

In [ ]:
d.did

In [ ]:
d.eid

In [ ]:
id_classes_b